In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, StackingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split, KFold

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from time import time

In [ ]:
train=pd.read_csv("../input/tabular-playground-series-sep-2021/train.csv")
test=pd.read_csv("../input/tabular-playground-series-sep-2021/test.csv")

train.shape, test.shape

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
id=test.id
train.drop("id", axis=1, inplace=True)
test.drop("id", axis=1, inplace=True)

In [ ]:
train.isnull().sum()

In [ ]:
sum(train.isnull().sum()>0)

In [ ]:
train["claim"].value_counts(dropna=False).plot(kind="bar")

In [ ]:
col=test.columns

imputer = SimpleImputer()
train[col]=imputer.fit_transform(train[col])
test[col]=imputer.transform(test[col])
train.head()

In [ ]:
scl=StandardScaler()
train[col]=scl.fit_transform(train[col])
test[col]=scl.transform(test[col])

In [ ]:
ind=np.random.permutation(len(train))
train=train.iloc[ind,].reset_index(drop=True)

In [ ]:
y=train["claim"].values
x=train.drop("claim", axis=1).values

In [ ]:
x_sample=x[:200000]
y_sample=y[:200000]

In [ ]:
x_train, x_test, y_train, y_test=train_test_split(x_sample,y_sample,test_size=0.3, random_state=4)

In [ ]:
def test_accuracy(models):
    for model in models:
        print("Model:{}".format(model))
        start_time=time()
        model.fit(x_train, y_train)
        pred=model.predict_proba(x_test)
        score=roc_auc_score(y_test,pred[:,1])
        print("ROC_AUC:{}".format(score))
        print("RunTime:{}".format(time()-start_time))
        print()
        print("*"*125)
        print()

In [ ]:
lr=LogisticRegression()
sgd=SGDClassifier()
rf=RandomForestClassifier(n_jobs=-1)
lgb=LGBMClassifier()
gbc=GradientBoostingClassifier()
cat=CatBoostClassifier(verbose=0)
xgb=XGBClassifier(eval_metric="logloss", n_jobs=-1, tree_method="hist")

In [ ]:
models=[lgb, cat, xgb]
test_accuracy(models)

In [ ]:
cat.fit(x,y)
pred=cat.predict_proba(test.values)

In [ ]:
pred=pred[:,1]

In [ ]:
prediction=pd.DataFrame()
prediction["id"]=id
prediction["claim"]=pred
prediction.head()

In [ ]:
prediction.to_csv("submission.csv", index=False)